In [1]:
import os
import pandas as pd
import zipfile
import pyodbc
import requests
import io
from datetime import datetime, timedelta
import sqlalchemy
import urllib
import warnings
warnings.filterwarnings('ignore')

In [2]:
NEMO = "DTE_EMISION"

URL = f"https://api.cammesa.com/pub-svc/public/"

method_id = {

    "findDocumentosByNemoRango?": 

}
method_id = "findDocumentosByNemoRango?" #ID
method_zip = "findAllAttachmentZipByNemoId?"

fecha_desde = "2024-03-31T00:00:00.000-03:00"
fecha_hasta = "2024-04-30T23:59:59.000-03:00"

In [3]:
url_doc_id = f"{URL}{method_id}fechadesde={fecha_desde}&fechahasta={fecha_hasta}&nemo={NEMO}"

In [4]:
try:
    with requests.get(url_doc_id) as response:
        if response.status_code == 200:
            PPO=response.json()
            doc_id = PPO[-1]['id']
        else:
            print("La solicitud falló con el código de estado:", response.status_code)
except requests.exceptions.RequestException as e:
    # Manejar la excepción
    print("Error al realizar la solicitud:", e)

In [5]:
url_zip = f"{URL}{method_zip}docId={doc_id}&nemo={NEMO}"

In [25]:
try:
    with requests.get(url_zip) as r:

        if r.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(r.content)) as primer_zip:
                z = primer_zip
                
                files_inside = z.namelist()
                DTE_name = z.namelist()[0]
                DTE_fecha = z.namelist()[0][:7]
                                
                os.mkdir(path=f"../.zips/{DTE_fecha}")
                z.extractall(path=f"../.zips/{DTE_fecha}") #Extraer todos los archivos del primer .zip

                os.mkdir(path=f"../.zips/{DTE_fecha}/{DTE_name}/")


                # Ruta al archivo .zip
                segundo_zip_path = f"../.zips/{DTE_fecha}/{DTE_name}/{DTE_name}"

                try:
                    # Verificar si el archivo .zip existe
                    if not os.path.exists(segundo_zip_path):
                        raise FileNotFoundError(f"El archivo .zip '{segundo_zip_path}' no existe.")

                    # Abrir el archivo .zip
                    with zipfile.ZipFile(segundo_zip_path, 'r') as zip_ref:
                        # Listar todos los archivos en el archivo .zip
                        lista_archivos = zip_ref.namelist()
                        print("Archivos en el .zip:")
                        for archivo in lista_archivos:
                            print(archivo)
                        
                        # Extraer todos los archivos a un directorio específico
                        ruta_extraccion = f"../.zips/{DTE_fecha}/{DTE_fecha}"
                        zip_ref.extractall(ruta_extraccion)
                        print(f"Todos los archivos extraídos exitosamente en '{ruta_extraccion}'.")
                        

                except FileNotFoundError as fnf_error:
                    print(fnf_error)
                except zipfile.BadZipFile as bzf_error:
                    print(f"Error con el archivo .zip: {bzf_error}")
                except Exception as error:
                    print(f"Error: {error}")

        else:
            print("La solicitud falló con el código de estado:", response.status_code)

except requests.exceptions.RequestException as e:
    # Manejar la excepción
    print("Error al realizar la solicitud:", e)



FileExistsError: [WinError 183] No se puede crear un archivo que ya existe: '../.zips/DTE2404/DTE2404.zip'